In [2]:
import string
import numpy as np
import os
from pickle import dump, load
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, get_file
from keras.layers import add, Input, Dense, LSTM, Embedding, Dropout
from keras.models import Model, load_model

#from tqdm import tqdm_notebook as tqdm
#tqdm().pandas()

In [3]:
#Load Files
def load_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [5]:
#Load Captions
def all_img_captions(filename):
    file = load_file(filename)
    captions = file.split('\n')
    descriptions = {}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [ caption ]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions
        

In [ ]:
#Cleaning Text
def cleaning_text(captions):
    table = str.maketrans('', '', string.punctuation)
    for img, caps in captions.items():
        for i, img_caption in enumerate(caps):
            img_caption = img_caption.replace("-","")
            desc = img_caption.split()
            desc = [word.lower() for word in desc]
            desc = [word.translate(table) for word in desc]
            desc = [word for word in desc if (len(word)) > 1]
            desc = [word for word in desc if (word.isalpha())]

            img_caption = ' '.join(desc)
            captions[img][i] = img_caption
    return captions

In [21]:
#Define Vocabolary
def text_vocabolary(descriptions):
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab

In [22]:
#Save Descriptions
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        lines.append(key + '\t' + ' '.join(desc_list))
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [23]:
#Load the Dataset
dataset_text = "D:\DataCamp Projects\LLM\Flickr8k_text"
dataset_images = "D:\DataCamp Projects\LLM\Flickr8k_Dataset"

filename = dataset_text + "/Flickr8k.token.txt"
descriptions = all_img_captions(filename)
print("Length of descriptions = ", len(descriptions))

Length of descriptions =  8092


In [24]:
#Clean save the descriptions

clean_descriptions = cleaning_text(descriptions)
vocabolary = text_vocabolary(clean_descriptions)
print("Length of Vocabolary = ", len(vocabolary))

save_descriptions(clean_descriptions, "descriptions.txt")

Length of Vocabolary =  8763
